In [1]:
from tensorflow.keras.models import load_model
def get_f1(y_true, y_pred):
    # Define your F1 metric calculation here
    pass
# Define the path to the model file
model_path = r"E:\new_model\finetuned_image_model.h5"
# Load the model with custom_objects argument
model = load_model(model_path, custom_objects={'get_f1': get_f1} )
# Print the model summary
#model.summary()

In [2]:
import json
import os
import tensorflow as tf
from PIL import Image
import numpy as np
from tqdm import tqdm

import os

def count_images_in_subdirs(folder_location):
    subdir_image_counts = []
    
    for subdir in os.listdir(folder_location):
        subdir_path = os.path.join(folder_location, subdir)
        
        if os.path.isdir(subdir_path):
            image_count = len([f for f in os.listdir(subdir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))])
            subdir_image_counts.append(image_count)
    
    return subdir_image_counts




img_list = []

def is_image(filename):
    try:
        with Image.open(filename) as img:
            return True
    except Exception as e:
        return False


def process_images(folder_location, target_size=(40, 40)):
    folders = os.listdir(folder_location)
    processed_images = []
    image_sizes = {}

    for folder in tqdm(folders):
        folder_path = os.path.join(folder_location, folder)
        images = os.listdir(folder_path)
        folder_image_sizes = []

        for image_name in images:
            image_path = os.path.join(folder_path, image_name)
            image = Image.open(image_path)

            # Get the original image size
            original_size = image.size
            folder_image_sizes.append(original_size)

            resized_image = tf.image.resize_with_crop_or_pad(
                tf.keras.preprocessing.image.img_to_array(image),
                target_size[0],
                target_size[1]
            )

            normalized_image = (resized_image - 127.5) / 127.5
            processed_images.append(normalized_image)
            img_list.append(image_name)

        image_sizes[folder] = folder_image_sizes

    processed_images = np.array(processed_images)
    return image_sizes, processed_images



def count_images_in_subdirs(folder_location):
    subdir_image_counts = []
    
    for subdir in os.listdir(folder_location):
        subdir_path = os.path.join(folder_location, subdir)
        
        if os.path.isdir(subdir_path):
            image_count = len([f for f in os.listdir(subdir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))])
            subdir_image_counts.append(image_count)
    
    return subdir_image_counts



def find_time(larvae):
    total_length = len(larvae)
    part_length = total_length // 3
    
    # Split the larvae array
    b = larvae[:part_length]
    m = larvae[part_length:2*part_length]
    e = larvae[2*part_length:]
    
    bb = np.mean(b, axis=0)
    mm = np.mean(m, axis=0)
    ee = np.mean(e, axis=0)
    values = [bb[1],mm[1],ee[1]]
    largest = values.index(max(values))
    return largest


def create_result_json(name, folder_location, image_sizes_dict, x, result1, count_of_ones,hc,time):
    # Convert NumPy arrays to lists
    x = x.tolist()
    y = len(result1)
    res_detailed_list = [item.tolist() for item in result1]
    result = {
        name: {
            "Subsample Name": folder_location,
            "Image Sizes": image_sizes_dict,
            "Floating Point Prediction": x,
            "Prediction": res_detailed_list,
            "Total Object Count": y,
            "Invasive Object Count": count_of_ones,
            "High COnfidence Invasive" : hc,
            "Most Likely time for Invasive" : time
        }
    }

    json_name = os.path.join(r"F:\JSONS", name + ".json")
    
    with open(json_name, 'w') as json_file:
        json.dump(result, json_file, indent=4)

import numpy as np


def model_predict_images(folder_location):
    
    image_counts = count_images_in_subdirs(folder_location)
    
    folders = folder_location.split("\\")

    folder = folders[-2]
    
    name = folder
    
    image_sizes_dict, processed_images = process_images(folder_location)

    x = model.predict(processed_images)
    
    hc_res = []
    result = []
    res_detailed = []
    time = []
    y = 0
    for i in image_counts:
        larvae = x[y:y+i]
        y = i+y
        average = np.mean(larvae, axis=0)
        pred = np.argmax(average)
        pred1 = pred
        if(pred==0):
            if(average[1]>.1):
                pred = 1
                l = find_time(larvae)
                time.append(l)
        result.append(pred1)
        hc_res.append(pred)
        res_detailed.append(average)

    count_of_ones = result.count(0)
    
    if not time:
        tn = 0
    else:
        tn = max(set(time), key = time.count)

    print("\n ------------------------ \n")

    print("Number of Total Organisms : ",len(result),"\n")
    print("Number of Invasive : ", count_of_ones,"\n")
    print("Number of High Confidence Invasive : ", hc_res.count(0),"\n")
    print("Number of Ostracod : ", result.count(2),"\n")
    print("Most likely time of veliger : ", tn,"\n")
    print("\n ------------------------ \n")

    create_result_json(name, folder_location,image_sizes_dict,x,result,count_of_ones,hc_res.count(1),tn)

In [34]:
import os

folder_location = r"D:\datasets\2.Veliger_Data\1.Samples 1 - 3\Samples 1 - 3\1 Below Lake Waco Deep 14_4 Concentrated"

# Get the list of all subdirectories
subdirectories = [f.path for f in os.scandir(folder_location) if f.is_dir()]

# Print the list of subdirectories
for subdirectory in tqdm(subdirectories):
    img_list = []
    model_predict_images(subdirectory+'\Images_001')

 36%|████████████████████████████▌                                                   | 199/558 [00:23<00:42,  8.48it/s]


 79%|██████████████████████████████████████████████████████████████▉                 | 439/558 [00:48<00:11, 10.20it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 558/558 [01:01<00:00,  9.07it/s]


683/683 [==============================] - 6s 9ms/step

 ------------------------ 

Number of Total Organisms :  558 

Number of Invasive :  36 

Number of High Confidence Invasive :  18 

Number of Ostracod :  2 

Most likely time of veliger :  2 


 ------------------------ 



 26%|████████████████████▊                                                           | 186/714 [00:30<01:15,  6.99it/s]


...

 67%|█████████████████████████████████████████████████████▎                          | 522/783 [01:56<01:53,  2.31it/s]


 85%|███████████████████████████████████████████████████████████████████▉            | 665/783 [02:57<00:32,  3.68it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 783/783 [03:51<00:00,  3.39it/s]


946/946 [==============================] - 2s 2ms/step

 ------------------------ 

Number of Total Organisms :  783 

Number of Invasive :  31 

Number of High Confidence Invasive :  15 

Number of Ostracod :  18 

Most likely time of veliger :  2 


 ------------------------ 



100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [09:14<00:00, 92.36s/it]


In [35]:
import os

folder_location = r"D:\datasets\2.Veliger_Data\1.Samples 1 - 3\Samples 1 - 3\2 Below Lake Waco Deep 28_8 Concentrated"

# Get the list of all subdirectories
subdirectories = [f.path for f in os.scandir(folder_location) if f.is_dir()]

# Print the list of subdirectories
for subdirectory in tqdm(subdirectories):
    img_list = []
    model_predict_images(subdirectory+'\Images_001')

 20%|████████████████                                                                | 135/671 [00:47<02:33,  3.48it/s]


 42%|█████████████████████████████████▌                                              | 281/671 [01:42<02:26,  2.66it/s]


 62%|█████████████████████████████████████████████████▍                              | 415/671 [02:35<02:14,  1.90it/s]


 83%|██████████████████████████████████████████████████████████████████▌             | 558/671 [03:22<00:40,  2.77it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 671/671 [03:51<00:00,  2.90it/s]


...

 68%|██████████████████████████████████████████████████████▍                         | 537/789 [03:49<01:41,  2.49it/s]


 85%|████████████████████████████████████████████████████████████████████▎           | 674/789 [04:38<00:34,  3.33it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 789/789 [05:31<00:00,  2.38it/s]


1057/1057 [==============================] - 2s 2ms/step

 ------------------------ 

Number of Total Organisms :  789 

Number of Invasive :  58 

Number of High Confidence Invasive :  29 

Number of Ostracod :  6 

Most likely time of veliger :  2 


 ------------------------ 



100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [18:03<00:00, 180.64s/it]


In [36]:
import os

folder_location = r"D:\datasets\2.Veliger_Data\2.Samples 4 - 6\Samples 4 - 6\4_39 Below Lake Waco Shallow 100 Gal"

# Get the list of all subdirectories
subdirectories = [f.path for f in os.scandir(folder_location) if f.is_dir()]

# Print the list of subdirectories
for subdirectory in tqdm(subdirectories):
    img_list = []
    model_predict_images(subdirectory+'\Images_001')

 14%|██████████▉                                                                     | 135/988 [01:18<04:06,  3.47it/s]


 28%|██████████████████████▋                                                         | 280/988 [02:16<04:28,  2.64it/s]


 42%|█████████████████████████████████▉                                              | 419/988 [03:08<04:23,  2.16it/s]


 56%|████████████████████████████████████████████▊                                   | 554/988 [04:18<04:34,  1.58it/s]


 70%|███████████████████████████████████████████████████████▊                        | 690/988 [05:30<02:13,  2.24it/s]


...

 86%|███████████████████████████████████████████████████████████████████           | 1329/1547 [16:24<03:32,  1.03it/s]


 94%|█████████████████████████████████████████████████████████████████████████▋    | 1461/1547 [18:16<01:20,  1.07it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 1547/1547 [19:32<00:00,  1.32it/s]


1931/1931 [==============================] - 6s 3ms/step

 ------------------------ 

Number of Total Organisms :  1547 

Number of Invasive :  106 

Number of High Confidence Invasive :  60 

Number of Ostracod :  36 

Most likely time of veliger :  2 


 ------------------------ 



100%|█████████████████████████████████████████████████████████████████████████████████| 6/6 [1:16:20<00:00, 763.35s/it]


In [4]:
import os

folder_location = r"D:\datasets\2.Veliger_Data\1.Samples 1 - 3\Samples 1 - 3\3 Below Lake Waco Deep 14_4 Gal"

# Get the list of all subdirectories
subdirectories = [f.path for f in os.scandir(folder_location) if f.is_dir()]

# Print the list of subdirectories
for subdirectory in tqdm(subdirectories):
    img_list = []
    model_predict_images(subdirectory+'\Images_001')

 11%|████████▋                                                                        | 88/821 [00:24<00:53, 13.80it/s]


 26%|█████████████████████▏                                                          | 217/821 [00:37<00:57, 10.43it/s]


 43%|██████████████████████████████████▏                                             | 351/821 [00:51<00:49,  9.49it/s]


 57%|█████████████████████████████████████████████▌                                  | 467/821 [01:05<00:38,  9.26it/s]


 72%|█████████████████████████████████████████████████████████▍                      | 589/821 [01:19<00:17, 12.93it/s]


...

 88%|█████████████████████████████████████████████████████████████████████▌         | 946/1075 [08:39<01:07,  1.91it/s]


 95%|██████████████████████████████████████████████████████████████████████████    | 1021/1075 [09:10<00:23,  2.30it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 1075/1075 [09:33<00:00,  1.87it/s]


1223/1223 [==============================] - 3s 2ms/step

 ------------------------ 

Number of Total Organisms :  1075 

Number of Invasive :  36 

Number of High Confidence Invasive :  20 

Number of Ostracod :  56 

Most likely time of veliger :  2 


 ------------------------ 



100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [57:25<00:00, 574.32s/it]
